In [1]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import datetime

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [3]:
with_categories = False
# Set a good base job name when building different models
# It will help in identifying trained models and endpoints
base_job_name = 'deepar-biketrain-with-dynamic-feat'

In [4]:
bucket = 'chandra-ml-sagemaker'
prefix = 'deepar/bikerental'

# This structure allows multiple training and test files for model development and testing
s3_data_path = "{}/{}/data_dynamic".format(bucket, prefix)
s3_output_path = "{}/{}/output".format(bucket, prefix)

In [5]:
s3_data_path,s3_output_path

('chandra-ml-sagemaker/deepar/bikerental/data_dynamic',
 'chandra-ml-sagemaker/deepar/bikerental/output')

In [6]:
# File name is referred as key name in S3
# Files stored in S3 are automatically replicated across
# three different availability zones in the region where the bucket was created.
# http://boto3.readthedocs.io/en/latest/guide/s3.html
def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f: # Read in binary mode
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [7]:
# Upload one or more training files and test files to S3
write_to_s3('train_dynamic_feat.json',bucket,'deepar/bikerental/data_dynamic/train/train_dynamic_feat.json')
write_to_s3('test_dynamic_feat.json',bucket,'deepar/bikerental/data_dynamic/test/test_dynamic_feat.json')

In [8]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [9]:
# We no longer have to maintain a mapping of container images by region
# Simply use the convenience method provided by sagemaker
# https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html
from sagemaker.amazon.amazon_estimator import get_image_uri
image_name = get_image_uri(boto3.Session().region_name, 'forecasting-deepar')

In [10]:
image_name

'522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:1'

In [11]:
freq='H' # Timeseries consists Hourly Data and we need to predict hourly rental count

# how far in the future predictions can be made
# 12 days worth of hourly forecast 
prediction_length = 288 

# aws recommends setting context same as prediction length as a starting point. 
# This controls how far in the past the network can see
context_length = 288

In [12]:
# Check Free Tier (if you are still under free-tier)
# At this time, m4.xlarge is offered as part of 2 months free tier
# https://aws.amazon.com/sagemaker/pricing/
# If you are outside of free-tier, you can also use ml.m5.xlarge  (newer generation instance)
# In this example, I am using ml.m5.xlarge for training

# Dynamic Feat - Using a large instance ml.c5.4xlarge = 16 CPU, 32 GB
# 'ml.c4.xlarge' -> 'ml.c5.4xlarge'. out of memory error with c4.xlarge
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c5.4xlarge',
    base_job_name=base_job_name,
    output_path="s3://" + s3_output_path
)

In [13]:
freq, context_length, prediction_length

('H', 288, 288)

In [14]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "cardinality" : "auto" if with_categories else ''
}

In [15]:
hyperparameters

{'time_freq': 'H',
 'epochs': '400',
 'early_stopping_patience': '40',
 'mini_batch_size': '64',
 'learning_rate': '5E-4',
 'context_length': '288',
 'prediction_length': '288',
 'cardinality': ''}

In [16]:
estimator.set_hyperparameters(**hyperparameters)

In [17]:
# Here, we are simply referring to train path and test path
# You can have multiple files in each path
# SageMaker will use all the files
data_channels = {
    "train": "s3://{}/train/".format(s3_data_path),
    "test": "s3://{}/test/".format(s3_data_path)
}

In [18]:
data_channels

{'train': 's3://chandra-ml-sagemaker/deepar/bikerental/data_dynamic/train/',
 'test': 's3://chandra-ml-sagemaker/deepar/bikerental/data_dynamic/test/'}

In [19]:
# This step takes around 35 minutes to train the model with m4.xlarge instance
estimator.fit(inputs=data_channels)

2020-04-23 23:53:26 Starting - Starting the training job...
2020-04-23 23:53:27 Starting - Launching requested ML instances......
2020-04-23 23:54:30 Starting - Preparing the instances for training...
2020-04-23 23:55:18 Downloading - Downloading input data...
2020-04-23 23:55:53 Training - Training image download completed. Training in progress..Arguments: train
[04/23/2020 23:55:54 INFO 139658144298816] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'cardinality': u'auto', u'likelihood': u'student-t', u'early_stopping_patience': u'

[04/23/2020 23:57:08 INFO 139658144298816] Epoch[2] Batch[10] avg_epoch_loss=3.619649
[04/23/2020 23:57:08 INFO 139658144298816] #quality_metric: host=algo-1, epoch=2, batch=10 train loss <loss>=3.58064804077
[04/23/2020 23:57:08 INFO 139658144298816] Epoch[2] Batch [10]#011Speed: 89.99 samples/sec#011loss=3.580648
[04/23/2020 23:57:08 INFO 139658144298816] processed a total of 670 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 16290.640115737915, "sum": 16290.640115737915, "min": 16290.640115737915}}, "EndTime": 1587686228.822237, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587686212.531545}

[04/23/2020 23:57:08 INFO 139658144298816] #throughput_metric: host=algo-1, train throughput=41.127719786 records/second
[04/23/2020 23:57:08 INFO 139658144298816] #progress_metric: host=algo-1, completed 0 % of epochs
[04/23/2020 23:57:08 INFO 139658144298816] #quality_metric: host=algo-1, epoch=2, train loss <loss>=3.619

[04/23/2020 23:58:24 INFO 139658144298816] Epoch[7] Batch[0] avg_epoch_loss=3.470984
[04/23/2020 23:58:24 INFO 139658144298816] #quality_metric: host=algo-1, epoch=7, batch=0 train loss <loss>=3.47098445892
[04/23/2020 23:58:27 INFO 139658144298816] Epoch[7] Batch[5] avg_epoch_loss=3.349670
[04/23/2020 23:58:27 INFO 139658144298816] #quality_metric: host=algo-1, epoch=7, batch=5 train loss <loss>=3.34966981411
[04/23/2020 23:58:27 INFO 139658144298816] Epoch[7] Batch [5]#011Speed: 92.81 samples/sec#011loss=3.349670
[04/23/2020 23:58:30 INFO 139658144298816] processed a total of 619 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 15855.191946029663, "sum": 15855.191946029663, "min": 15855.191946029663}}, "EndTime": 1587686310.630789, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587686294.775542}

[04/23/2020 23:58:30 INFO 139658144298816] #throughput_metric: host=algo-1, train throughput=39.0406372973 records/secon

[04/23/2020 23:59:46 INFO 139658144298816] Epoch[12] Batch[0] avg_epoch_loss=2.983557
[04/23/2020 23:59:46 INFO 139658144298816] #quality_metric: host=algo-1, epoch=12, batch=0 train loss <loss>=2.98355698586
[04/23/2020 23:59:49 INFO 139658144298816] Epoch[12] Batch[5] avg_epoch_loss=3.142137
[04/23/2020 23:59:49 INFO 139658144298816] #quality_metric: host=algo-1, epoch=12, batch=5 train loss <loss>=3.14213665326
[04/23/2020 23:59:49 INFO 139658144298816] Epoch[12] Batch [5]#011Speed: 91.97 samples/sec#011loss=3.142137
[04/23/2020 23:59:52 INFO 139658144298816] processed a total of 640 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 15786.023139953613, "sum": 15786.023139953613, "min": 15786.023139953613}}, "EndTime": 1587686392.66927, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587686376.883202}

[04/23/2020 23:59:52 INFO 139658144298816] #throughput_metric: host=algo-1, train throughput=40.5419856264 records/s

[04/24/2020 00:01:07 INFO 139658144298816] Epoch[17] Batch[0] avg_epoch_loss=2.921350
[04/24/2020 00:01:07 INFO 139658144298816] #quality_metric: host=algo-1, epoch=17, batch=0 train loss <loss>=2.92135000229
[04/24/2020 00:01:11 INFO 139658144298816] Epoch[17] Batch[5] avg_epoch_loss=2.923197
[04/24/2020 00:01:11 INFO 139658144298816] #quality_metric: host=algo-1, epoch=17, batch=5 train loss <loss>=2.92319746812
[04/24/2020 00:01:11 INFO 139658144298816] Epoch[17] Batch [5]#011Speed: 90.97 samples/sec#011loss=2.923197
[04/24/2020 00:01:14 INFO 139658144298816] Epoch[17] Batch[10] avg_epoch_loss=2.855245
[04/24/2020 00:01:14 INFO 139658144298816] #quality_metric: host=algo-1, epoch=17, batch=10 train loss <loss>=2.77370109558
[04/24/2020 00:01:14 INFO 139658144298816] Epoch[17] Batch [10]#011Speed: 89.94 samples/sec#011loss=2.773701
[04/24/2020 00:01:14 INFO 139658144298816] processed a total of 648 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 16558.753967285156, 

...

[04/24/2020 00:48:26 INFO 139658144298816] Epoch[193] Batch[0] avg_epoch_loss=2.384459
[04/24/2020 00:48:26 INFO 139658144298816] #quality_metric: host=algo-1, epoch=193, batch=0 train loss <loss>=2.38445949554
[04/24/2020 00:48:29 INFO 139658144298816] Epoch[193] Batch[5] avg_epoch_loss=2.471065
[04/24/2020 00:48:29 INFO 139658144298816] #quality_metric: host=algo-1, epoch=193, batch=5 train loss <loss>=2.47106472651
[04/24/2020 00:48:29 INFO 139658144298816] Epoch[193] Batch [5]#011Speed: 92.75 samples/sec#011loss=2.471065
[04/24/2020 00:48:33 INFO 139658144298816] Epoch[193] Batch[10] avg_epoch_loss=2.415948
[04/24/2020 00:48:33 INFO 139658144298816] #quality_metric: host=algo-1, epoch=193, batch=10 train loss <loss>=2.34980788231
[04/24/2020 00:48:33 INFO 139658144298816] Epoch[193] Batch [10]#011Speed: 90.03 samples/sec#011loss=2.349808
[04/24/2020 00:48:33 INFO 139658144298816] processed a total of 648 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 16454.603910

[04/24/2020 00:49:48 INFO 139658144298816] Epoch[198] Batch[0] avg_epoch_loss=2.468083
[04/24/2020 00:49:48 INFO 139658144298816] #quality_metric: host=algo-1, epoch=198, batch=0 train loss <loss>=2.46808338165
[04/24/2020 00:49:52 INFO 139658144298816] Epoch[198] Batch[5] avg_epoch_loss=2.445148
[04/24/2020 00:49:52 INFO 139658144298816] #quality_metric: host=algo-1, epoch=198, batch=5 train loss <loss>=2.4451482296
[04/24/2020 00:49:52 INFO 139658144298816] Epoch[198] Batch [5]#011Speed: 94.08 samples/sec#011loss=2.445148
[04/24/2020 00:49:55 INFO 139658144298816] Epoch[198] Batch[10] avg_epoch_loss=2.420891
[04/24/2020 00:49:55 INFO 139658144298816] #quality_metric: host=algo-1, epoch=198, batch=10 train loss <loss>=2.39178142548
[04/24/2020 00:49:55 INFO 139658144298816] Epoch[198] Batch [10]#011Speed: 89.69 samples/sec#011loss=2.391781
[04/24/2020 00:49:55 INFO 139658144298816] processed a total of 682 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 16471.5249538

[04/24/2020 00:51:11 INFO 139658144298816] Epoch[203] Batch[0] avg_epoch_loss=2.531775
[04/24/2020 00:51:11 INFO 139658144298816] #quality_metric: host=algo-1, epoch=203, batch=0 train loss <loss>=2.53177452087
[04/24/2020 00:51:14 INFO 139658144298816] Epoch[203] Batch[5] avg_epoch_loss=2.485362
[04/24/2020 00:51:14 INFO 139658144298816] #quality_metric: host=algo-1, epoch=203, batch=5 train loss <loss>=2.48536169529
[04/24/2020 00:51:14 INFO 139658144298816] Epoch[203] Batch [5]#011Speed: 93.18 samples/sec#011loss=2.485362
[04/24/2020 00:51:18 INFO 139658144298816] Epoch[203] Batch[10] avg_epoch_loss=2.444990
[04/24/2020 00:51:18 INFO 139658144298816] #quality_metric: host=algo-1, epoch=203, batch=10 train loss <loss>=2.39654436111
[04/24/2020 00:51:18 INFO 139658144298816] Epoch[203] Batch [10]#011Speed: 89.47 samples/sec#011loss=2.396544
[04/24/2020 00:51:18 INFO 139658144298816] processed a total of 648 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 16474.951028

[04/24/2020 00:52:46 INFO 139658144298816] Epoch[209] Batch[0] avg_epoch_loss=2.592492
[04/24/2020 00:52:46 INFO 139658144298816] #quality_metric: host=algo-1, epoch=209, batch=0 train loss <loss>=2.59249162674
[04/24/2020 00:52:50 INFO 139658144298816] Epoch[209] Batch[5] avg_epoch_loss=2.581632
[04/24/2020 00:52:50 INFO 139658144298816] #quality_metric: host=algo-1, epoch=209, batch=5 train loss <loss>=2.58163154125
[04/24/2020 00:52:50 INFO 139658144298816] Epoch[209] Batch [5]#011Speed: 94.05 samples/sec#011loss=2.581632
[04/24/2020 00:52:53 INFO 139658144298816] processed a total of 624 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 15639.203071594238, "sum": 15639.203071594238, "min": 15639.203071594238}}, "EndTime": 1587689573.051054, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1587689557.411584}

[04/24/2020 00:52:53 INFO 139658144298816] #throughput_metric: host=algo-1, train throughput=39.8994940573 rec


2020-04-24 00:54:16 Completed - Training job completed
Training seconds: 3538
Billable seconds: 3538


In [24]:
job_name = estimator.latest_training_job.name

In [25]:
print ('job name: {0}'.format(job_name))

job name: deepar-biketrain-with-dynamic-feat-2020-04-23-23-53-26-466


In [26]:
# Create an endpoint for real-time predictions
endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    deployment_image=image_name,
    role=role
)

---------------!

In [ ]:
print ('endpoint name: {0}'.format(endpoint_name))

In [ ]:
# Don't forget to terminate the end point after completing the demo
# Otherwise, you account will accumulate hourly charges

# you can delete from sagemaker management console or through command line or throught code

#sagemaker_session.delete_endpoint(endpoint_name)